In [ ]:
import sys
sys.path.append('../Tools/')

import datacube
from datacube.utils import cog
from dea_tools.dask import create_local_dask_cluster

In [ ]:
client = create_local_dask_cluster(return_client=True)

In [ ]:
dc = datacube.Datacube(app="DEA_Sentinel2_Surface_Reflectance")
#dc_products = dc.list_products()
#dc_products.loc[['ga_s2am_ard_3', 'ga_s2bm_ard_3']]
#dc_measurements = dc.list_measurements()
#dc_measurements.loc[['ga_s2am_ard_3']]

In [ ]:
sites = ['Fowlers_Gap', 'Witchelina']
eastings_northings = [[(141.4, 142.0), (-30.8, -31.2)],
                      [(137.5, 138.3), (-29.7, -30.6)]]
dates = [('2023-01-05', '2023-01-07'),
         ('2023-02-07', '2023-02-09')]

for i, e_n in enumerate(eastings_northings):
    s2_ds = dc.load(product=['ga_s2am_ard_3', 'ga_s2bm_ard_3'],
                    measurements=['blue', 'green', 'red', 'red_edge_1', 'red_edge_2', 'red_edge_3', 'nir_1', 'swir_2', 'swir_3'],
                    x=e_n[0],
                    y=e_n[1],
                    time=dates[i],
                    output_crs='EPSG:32754',
                    resolution=(-10, 10),
                    resampling='cubic',
                    dask_chunks=dict(x=1024, y=1024))
    
    for t in range(len(s2_ds.time)):
        date = s2_ds.isel(time=t).time.dt.strftime('%Y%m%d').data
        ds_delayed = cog.write_cog(s2_ds.isel(time=t).to_array(), f's2_ard_%s_%s_%i.tif'%(sites[i], date, t), overwrite=True, nodata=-999)
        ds_delayed.compute()